In [85]:
def get_businesses(query, api_key, min_rating=0.0, max_results=30):
    import requests
    api_base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query='
    url = api_base_url + query + '&radius=1500&key=' + api_key
    url = url.replace(' ', '+')
    response = requests.get(url)
    
    # instantiates empty list of businesses
    businesses = []
    
    # variable to keep track of whether we should continue to search and append business tuples to the list of businesses
    go = True
    
    # keeps track of how many times we traverse the while loop --> important to see if we need to access a new page or not
    iteration = 0
    
    # while loop continues as long as go is True. This was done to make sure that the new url could be accessed for the
    # next page, so that the for loop could then iterate through the next page and output more results
    while go == True:
        
        # if it is not the first iteration through the while loop and if the length of the list is still 
        # less than the user specified number of results then check then it goes to the next page
        # if it has a next_page_token
        if iteration != 0 and len(businesses) <= max_results:
            if 'next_page_token' in response.json():
                new_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken='+ response.json()['next_page_token'] + '&radius=1500&key=' + api_key
                response = requests.get(new_url)
                
                # professor's code to make sure that there is no invalid request accessing the api
                while response.json().get('status') != 'OK':
                    from time import sleep
                    from random import random
                    sleep(random())
                    response = requests.get(new_url)
            else:
                go = False
                  
        # iterates through each element of the json results
        for result in response.json()['results']:

            # checks to see if we already have reached the number of businesses to be outputted
            if len(businesses) == max_results:
                go = False
                break
            
            # if the new result's first name is already in the list then terminate the code
            # this is done to make sure a repeat page's data is not put into the list
            if (any(result['name'] in i for i in businesses)):
                go = False
                print("There are no more results so this is your list of businesses: ")
                break

            # if the rating isn't up to the min rating then it skips the iteration
            if result['rating'] <= min_rating:
                continue

            # takes the desired string and saves it to a variable to be added to the tuple in the list
            # for prices, checks to see if the parameter exists and if it does then saves it, otherwise saves it as None
            business_name = result['name']
            business_address = result['formatted_address']

            # dictionary to save if the business is currently open or not
            hours = {}
            hours['open_now'] = result['opening_hours']['open_now']

            if 'price_level' not in result:
                price_level = None
            else:
                price_level = result['price_level']

            rating = result['rating']
            
            # appends business tuple to the list
            businesses.append((business_name, business_address, hours, price_level, rating))
        
        # adds 1 to the number of iterations through the while loop
        iteration +=1
        
    return businesses

In [86]:
get_businesses('restaurants near Columbia University', api_key, min_rating = 4.0, max_results = 30)

[('Community Food & Juice',
  '2893 Broadway, New York, NY 10025, USA',
  {'open_now': True},
  2,
  4.2),
 ('John Jay Dining Hall',
  '519 W 114th St, New York, NY 10027, USA',
  {'open_now': True},
  None,
  4.1),
 ("Friedman's",
  '1187 Amsterdam Ave, New York, NY 10027, USA',
  {'open_now': True},
  2,
  4.3),
 ("Mel's Burger Bar",
  '2850 Broadway between 110th and, 111th, New York, NY 10025, United States',
  {'open_now': True},
  2,
  4.1),
 ("Brownie's Cafe",
  '1172 Amsterdam Ave, New York, NY 10027, USA',
  {'open_now': False},
  1,
  4.6),
 ('Symposium',
  '544 W 113th St, New York, NY 10025, USA',
  {'open_now': True},
  2,
  4.2),
 ('Shake Shack',
  '2957 Broadway, New York, NY 10025, USA',
  {'open_now': True},
  2,
  4.2),
 ('Dig Inn',
  '2884 Broadway, New York, NY 10025, USA',
  {'open_now': True},
  1,
  4.4),
 ('Roti Roll Bombay Frankie',
  '994 Amsterdam Ave, New York, NY 10025, USA',
  {'open_now': True},
  1,
  4.5),
 ('Junzi Kitchen',
  '2896 Broadway, New York, 